In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import pickle
import os

2025-04-10 18:27:42.521515: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-10 18:27:42.714366: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744309662.780397    5482 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744309662.798254    5482 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1744309662.941948    5482 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [2]:
print("Chargement du dataset Plant Village...")
# Charger le dataset Plant Village
dataset_name = "plant_village"
builder = tfds.builder(dataset_name)
builder.download_and_prepare()
dataset_info = builder.info


Chargement du dataset Plant Village...


In [ ]:
# Récupération des classes et mapping
class_names = dataset_info.features['label'].names
num_classes = dataset_info.features['label'].num_classes
print(f"Nombre de classes: {num_classes}")
print(f"Noms des classes: {class_names}")

Nombre de classes: 38
Noms des classes: ['Apple___Apple_scab', 'Apple___Black_rot', 'Apple___Cedar_apple_rust', 'Apple___healthy', 'Blueberry___healthy', 'Cherry___healthy', 'Cherry___Powdery_mildew', 'Corn___Cercospora_leaf_spot Gray_leaf_spot', 'Corn___Common_rust', 'Corn___healthy', 'Corn___Northern_Leaf_Blight', 'Grape___Black_rot', 'Grape___Esca_(Black_Measles)', 'Grape___healthy', 'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)', 'Orange___Haunglongbing_(Citrus_greening)', 'Peach___Bacterial_spot', 'Peach___healthy', 'Pepper,_bell___Bacterial_spot', 'Pepper,_bell___healthy', 'Potato___Early_blight', 'Potato___healthy', 'Potato___Late_blight', 'Raspberry___healthy', 'Soybean___healthy', 'Squash___Powdery_mildew', 'Strawberry___healthy', 'Strawberry___Leaf_scorch', 'Tomato___Bacterial_spot', 'Tomato___Early_blight', 'Tomato___healthy', 'Tomato___Late_blight', 'Tomato___Leaf_Mold', 'Tomato___Septoria_leaf_spot', 'Tomato___Spider_mites Two-spotted_spider_mite', 'Tomato___Target_Spot', 'T

In [ ]:
class_mapping = {i: name for i, name in enumerate(class_names)}

In [5]:
class_mapping

{0: 'Apple___Apple_scab',
 1: 'Apple___Black_rot',
 2: 'Apple___Cedar_apple_rust',
 3: 'Apple___healthy',
 4: 'Blueberry___healthy',
 5: 'Cherry___healthy',
 6: 'Cherry___Powdery_mildew',
 7: 'Corn___Cercospora_leaf_spot Gray_leaf_spot',
 8: 'Corn___Common_rust',
 9: 'Corn___healthy',
 10: 'Corn___Northern_Leaf_Blight',
 11: 'Grape___Black_rot',
 12: 'Grape___Esca_(Black_Measles)',
 13: 'Grape___healthy',
 14: 'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)',
 15: 'Orange___Haunglongbing_(Citrus_greening)',
 16: 'Peach___Bacterial_spot',
 17: 'Peach___healthy',
 18: 'Pepper,_bell___Bacterial_spot',
 19: 'Pepper,_bell___healthy',
 20: 'Potato___Early_blight',
 21: 'Potato___healthy',
 22: 'Potato___Late_blight',
 23: 'Raspberry___healthy',
 24: 'Soybean___healthy',
 25: 'Squash___Powdery_mildew',
 26: 'Strawberry___healthy',
 27: 'Strawberry___Leaf_scorch',
 28: 'Tomato___Bacterial_spot',
 29: 'Tomato___Early_blight',
 30: 'Tomato___healthy',
 31: 'Tomato___Late_blight',
 32: 'Tomato___Leaf

In [6]:
# Sauvegarde du mapping des classes
with open('class_mapping.pkl', 'wb') as f:
    pickle.dump(class_mapping, f)
print("Mapping des classes sauvegardé dans 'class_mapping.pkl'")

Mapping des classes sauvegardé dans 'class_mapping.pkl'


In [ ]:
# Récupération des données (train/validation)
print("Préparation des datasets...")
train_dataset = builder.as_dataset(split=tfds.core.ReadInstruction('train', to=80, unit='%'), as_supervised=True)
val_dataset = builder.as_dataset(split=tfds.core.ReadInstruction('train', from_=80, unit='%'), as_supervised=True)

Préparation des datasets...


In [ ]:
# Prétraitement pour l'entraînement des données
def preprocess_train_image(image, label):
    
    # Redimensionner l'image à une taille fixe
    image = tf.image.resize(image, (128, 128))
    
    # Appliquer l'augmentation
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_brightness(image, max_delta=0.2)
    
    # Normaliser les valeurs
    image = tf.cast(image, tf.float32) / 255.0
    return image, label

In [ ]:
#Prétraitement pour la validation sans augmentation
def preprocess_val_image(image, label):
    # Redimensionner l'image à une taille fixe
    image = tf.image.resize(image, (128, 128))
    # Normaliser les valeurs de pixels
    image = tf.cast(image, tf.float32) / 255.0
    return image, label


In [ ]:
# Appliquer le prétraitement et préparer les datasets
train_dataset = train_dataset.map(preprocess_train_image)
train_dataset = train_dataset.shuffle(32).batch(1000).prefetch(tf.data.AUTOTUNE)

val_dataset = val_dataset.map(preprocess_val_image)
val_dataset = val_dataset.batch(32).prefetch(tf.data.AUTOTUNE)


In [ ]:
# Définition du modèle CNN
print("Construction du modèle...")
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(num_classes, activation='softmax')
])

Construction du modèle...


In [15]:
# Compilation du modèle
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [16]:
# Afficher un résumé du modèle
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_3 (Conv2D)               │ (None, 126, 126, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 61, 61, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 28, 28, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 14, 14, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │     3,211,392 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 38)             │         4,902 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,309,542 (12.62 MB)

 Trainable params: 3,309,542 (12.62 MB)

 Non-trainable params: 0 (0.00 B)

In [18]:
# Entraînement du modèle
print("Début de l'entraînement...")
history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=20,
    verbose=1
)

Début de l'entraînement...
Epoch 1/20


2025-04-10 18:31:13.931205: W tensorflow/core/kernels/data/prefetch_autotuner.cc:52] Prefetch autotuner tried to allocate 196616000 bytes after encountering the first element of size 196616000 bytes.This already causes the autotune ram budget to be exceeded. To stay within the ram budget, either increase the ram budget or reduce element size


44/44 ━━━━━━━━━━━━━━━━━━━━ 212s 5s/step - accuracy: 0.2062 - loss: 3.1258 - val_accuracy: 0.5282 - val_loss: 1.7697
Epoch 2/20
44/44 ━━━━━━━━━━━━━━━━━━━━ 190s 4s/step - accuracy: 0.4677 - loss: 1.9529 - val_accuracy: 0.6767 - val_loss: 1.1694
Epoch 3/20
44/44 ━━━━━━━━━━━━━━━━━━━━ 209s 5s/step - accuracy: 0.5899 - loss: 1.4514 - val_accuracy: 0.7479 - val_loss: 0.8728
Epoch 4/20


2025-04-10 18:41:25.493914: W tensorflow/core/kernels/data/prefetch_autotuner.cc:52] Prefetch autotuner tried to allocate 196616000 bytes after encountering the first element of size 196616000 bytes.This already causes the autotune ram budget to be exceeded. To stay within the ram budget, either increase the ram budget or reduce element size


44/44 ━━━━━━━━━━━━━━━━━━━━ 207s 5s/step - accuracy: 0.6526 - loss: 1.1975 - val_accuracy: 0.8000 - val_loss: 0.6909
Epoch 5/20


...

44/44 ━━━━━━━━━━━━━━━━━━━━ 212s 5s/step - accuracy: 0.8622 - loss: 0.4268 - val_accuracy: 0.9226 - val_loss: 0.2475
Epoch 19/20


2025-04-10 19:33:45.967674: W tensorflow/core/kernels/data/prefetch_autotuner.cc:52] Prefetch autotuner tried to allocate 196616000 bytes after encountering the first element of size 196616000 bytes.This already causes the autotune ram budget to be exceeded. To stay within the ram budget, either increase the ram budget or reduce element size


44/44 ━━━━━━━━━━━━━━━━━━━━ 214s 5s/step - accuracy: 0.8655 - loss: 0.4163 - val_accuracy: 0.9222 - val_loss: 0.2573
Epoch 20/20


2025-04-10 19:37:20.410710: W tensorflow/core/kernels/data/prefetch_autotuner.cc:52] Prefetch autotuner tried to allocate 196616000 bytes after encountering the first element of size 196616000 bytes.This already causes the autotune ram budget to be exceeded. To stay within the ram budget, either increase the ram budget or reduce element size


44/44 ━━━━━━━━━━━━━━━━━━━━ 231s 5s/step - accuracy: 0.8694 - loss: 0.4046 - val_accuracy: 0.9253 - val_loss: 0.2474


In [19]:
# Sauvegarde du modèle
model_p = 'plant_disease_model.h5'
model.save(model_p)
print(f"Modèle sauvegardé sous: {model_p}")

Modèle sauvegardé sous: plant_disease_model.h5


In [ ]:
# Configuration pour la prédiction
model_config = {
    'img_height': 128,
    'img_width': 128,
    'img_channels': 32,
    'class_mapping_file': 'class_mapping.pkl',
    'model_file': model_p
}


In [21]:
# Sauvegarder la configuration
with open('model_config.pkl', 'wb') as f:
    pickle.dump(model_config, f)
print("Configuration du modèle sauvegardée dans 'model_config.pkl'")

print("Entraînement terminé!")

Configuration du modèle sauvegardée dans 'model_config.pkl'
Entraînement terminé!
